# 1 Processing

## Praperation

In [1]:
import os
HOME = os.getcwd()
print(HOME)

os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

/home/jpan/xAI_Masterthesis_Pan/Code/Detection/YOLO/202407_Mixed_tests


In [2]:
#clearml mac
# %env CLEARML_WEB_HOST=https://app.clear.ml
# %env CLEARML_API_HOST=https://api.clear.ml
# %env CLEARML_FILES_HOST=https://files.clear.ml
# %env CLEARML_API_ACCESS_KEY=QO1VBBX9J2S2VYILQTGI
# %env CLEARML_API_SECRET_KEY=ERuc1S6o5SirQGugvYXDFjH9b9aNi0u8S3rpALzXMa8YPSLDMW

# clearml win
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=Z4YDBU13VPOFHBHF8667
%env CLEARML_API_SECRET_KEY=JMzvRXn76AT83WuFJfS0FBGCY8c5TccbH5XboTYztrWqwzDdyn

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=Z4YDBU13VPOFHBHF8667
env: CLEARML_API_SECRET_KEY=JMzvRXn76AT83WuFJfS0FBGCY8c5TccbH5XboTYztrWqwzDdyn


In [3]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.41 🚀 Python-3.12.2 torch-2.3.1+cu121 CUDA:0 (NVIDIA RTX A5000, 24241MiB)
Setup complete ✅ (24 CPUs, 125.5 GB RAM, 320.5/459.6 GB disk)


/home/jpan/miniconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


## Config

In [4]:
# read config file
import yaml

def load_config(config_file):
    with open(config_file, 'r') as f:
        config = yaml.safe_load(f)
    return config

config = load_config('Mixed_all_config_1.yaml')

print(config)

# project
project = config['project']
name = config['name']
exist_ok = config['exist_ok']

# dataset
data_folder = config['data_server']
data = config['data_yaml']
test_data = config['test_data']
imgsz = config['imgsz']

# model
model = config['model']
task = config['task']
mode = config['mode']
# device = config['device_mac']
device = config['device_win']

# Training parameters
epochs = config['epochs']
batch = config['batch']
lr0 = config['lr0']
cos_lr = config['cos_lr']
weight_decay = config['weight_decay']
dropout = config['dropout']
optimizer = config['optimizer']
momentum = config['momentum']
seed = config['seed']

# Output configuration
# save = config['save']
# save_period = config['save_period']
plots = config['plots']
test_output = config['test_output']


{'project': 'Uni-Wood', 'name': 'Mixed_YOLO_all_test_1', 'exist_ok': False, 'data_server': '/mnt/data/backup_pan/Dataset/Mixed_datasets/Det-dominik_v_1-v2-sf_v1.v2i.yolov8', 'data_yaml': '/mnt/data/backup_pan/Dataset/Mixed_datasets/Det-dominik_v_1-v2-sf_v1.v2i.yolov8/data.yaml', 'test_data': '/path/to/test_data.txt', 'imgsz': 640, 'model': 'yolov10m.yaml', 'task': 'detect', 'mode': 'train', 'device_mac': 'mps', 'device_win': 0, 'epochs': 1, 'batch': 16, 'lr0': 0.001, 'cos_lr': True, 'weight_decay': 0.0005, 'dropout': 0.0, 'optimizer': 'Adam', 'momentum': 0.9, 'seed': 42, 'save': False, 'save_period': -1, 'plots': True, 'test_output': '/path/to/test_output'}


## Split data

In [ ]:
import os
import shutil
import random

train_dir = os.path.join(data_folder, "train")
val_dir = os.path.join(data_folder, "valid")
test_dir = os.path.join(data_folder, "test")

os.makedirs(os.path.join(val_dir, "images"), exist_ok=True)
os.makedirs(os.path.join(val_dir, "labels"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "images"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "labels"), exist_ok=True)

val_ratio = 0.1
test_ratio = 0.05

random.seed(seed)

image_files = os.listdir(os.path.join(train_dir, "images"))
label_files = os.listdir(os.path.join(train_dir, "labels"))

image_files.sort()
label_files.sort()

data_pairs = list(zip(image_files, label_files))

random.shuffle(data_pairs)

total_files = len(data_pairs)
val_count = int(total_files * val_ratio)
test_count = int(total_files * test_ratio)

val_data = data_pairs[:val_count]
test_data = data_pairs[val_count:val_count+test_count]
train_data = data_pairs[val_count+test_count:]



In [ ]:
def move_files(data, src_dir, dest_dir):
    for img_file, lbl_file in data:
        shutil.move(os.path.join(src_dir, "images", img_file), os.path.join(dest_dir, "images", img_file))
        shutil.move(os.path.join(src_dir, "labels", lbl_file), os.path.join(dest_dir, "labels", lbl_file))

move_files(val_data, train_dir, val_dir)
move_files(test_data, train_dir, test_dir)

print("Split done!")

In [ ]:
def count_files(directory):
    images_count = len(os.listdir(os.path.join(directory, "images")))
    labels_count = len(os.listdir(os.path.join(directory, "labels")))
    return images_count, labels_count

train_images_count, train_labels_count = count_files(train_dir)
val_images_count, val_labels_count = count_files(val_dir)
test_images_count, test_labels_count = count_files(test_dir)

print(f"Train set: {train_images_count} images, {train_labels_count} labels")
print(f"Validation set: {val_images_count} images, {val_labels_count} labels")
print(f"Test set: {test_images_count} images, {test_labels_count} labels")

In [ ]:
def move_back_files(src_dir, dest_dir):
    for file_name in os.listdir(os.path.join(src_dir, "images")):
        shutil.move(os.path.join(src_dir, "images", file_name), os.path.join(dest_dir, "images", file_name))
    for file_name in os.listdir(os.path.join(src_dir, "labels")):
        shutil.move(os.path.join(src_dir, "labels", file_name), os.path.join(dest_dir, "labels", file_name))

move_back_files(val_dir, train_dir)
move_back_files(test_dir, train_dir)

print("Recovered!")


## Clearml

In [5]:
from clearml import Task

#Clear ML Initialization
cl_task = Task.init(project_name=project,task_name=name)
logger = cl_task.get_logger()

ClearML Task: created new task id=93791013d21249788478a2f39c080d63
2024-07-17 11:21:56,624 - clearml.Task - INFO - Storing jupyter notebook directly as code


# 2 Training

In [ ]:
from ultralytics import YOLO

model = YOLO(model)

results = model.train(project=project, name=name, exist_ok=exist_ok, data=data, imgsz=imgsz, task=task, device=device, epochs=epochs, batch=batch, lr0=lr0, cos_lr=cos_lr, weight_decay=weight_decay, dropout=dropout, optimizer=optimizer, momentum=momentum, seed=seed, plots=plots)

cl_task.close()

# 3 testing

## 3.1 path

In [ ]:
from ultralytics import YOLO
import cv2
import os

model_path = "/Users/holmes/Documents/UNI-Bamberg/Arbeiten/Uni_Bamberg_KDWT_Holzprojekt/Project_process/P2_Experiments/202403_Schweinfurt/Uni-Wood/Schweinfurt-Yolo-Test-4/weights/best.pt"

image_folder = "/Users/holmes/Desktop/probe"

save_path = "/Users/holmes/Desktop/probe"



## 3.2 resize

In [ ]:
# Resize images
resized_image_folder = os.path.join(image_folder, "resized")
os.makedirs(resized_image_folder, exist_ok=True)

for file in os.listdir(image_folder):
    if file.endswith(('.jpg', '.JPG', '.jpeg', '.png', '.bmp')):
        img_path = os.path.join(image_folder, file)
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, (640, 640))
        resized_img_path = os.path.join(resized_image_folder, os.path.splitext(file)[0] + "_resize" + os.path.splitext(file)[1])
        cv2.imwrite(resized_img_path, img_resized)
image_folder = resized_image_folder


## 3.3 test

In [ ]:
model = YOLO(model_path)

if not os.path.exists(save_path):
    os.makedirs(save_path)

image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(('.jpg', '.jpeg', '.JPG', '.png', '.bmp'))]

for img_path in image_files:
    results = model(img_path)
    img = cv2.imread(img_path)

    for result in results:
        boxes = result.boxes.cpu().numpy()
        for box in boxes:
            # box contains prediction information
            confidence = box.conf[0]
            if confidence > 0.4:
                xyxy = box.xyxy[0].astype(int)
                confidence_text = f"{confidence:.2f}"

                cv2.rectangle(img, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 0, 255), 2)
                cv2.putText(img, confidence_text, (xyxy[0], xyxy[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

    unique_save_path = os.path.join(save_path, os.path.basename(img_path))
    cv2.imwrite(unique_save_path, img)
    print(f"Image saved: {unique_save_path}")
